In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install git+https://github.com/huggingface/transformers.git@master
!pip install git+https://github.com/huggingface/datasets.git@master
!pip install sentencepiece

  Cloning https://github.com/huggingface/transformers.git (to revision master) to /tmp/pip-req-build-7lem_yew
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-7lem_yew
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: filename=transformers-4.6.0.dev0-cp37-none-any.whl size=2112513 sha256=66284a893cab115866da305c354c84c6948c5d801561d3da483bcc4be868f19f
  Stored in directory: /tmp/pip-ephem-wheel-cache-lnbt44fn/wheels/03/01/00/f2c9020459e177a2729b25f0b9628d95bb967727d71a118170
Successfully built transformers
  Cloning https://github.com/huggingface/datasets.git (to revision master) to /tmp/pip-req-build-31xf530o
  Running command git clone -q https://github.com/huggingface/datasets.git /tmp/pip-req-build-31xf530o
  Created wheel for datasets: filename=datasets-1.5.0.dev0-cp37-none-any.whl size=199485 sha256=37f142e87e506bb4a4ae0cbe

In [ ]:
from datasets import load_dataset
from transformers import ( XLMProphetNetForConditionalGeneration, XLMProphetNetTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer)
import torch
from torch.utils.data import random_split

In [ ]:
!wget -c "http://www.cfilt.iitb.ac.in/iitb_parallel/iitb_corpus_download/parallel.tgz"

--2021-04-20 15:01:38--  http://www.cfilt.iitb.ac.in/iitb_parallel/iitb_corpus_download/parallel.tgz
Resolving www.cfilt.iitb.ac.in (www.cfilt.iitb.ac.in)... 103.21.127.134
Connecting to www.cfilt.iitb.ac.in (www.cfilt.iitb.ac.in)|103.21.127.134|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cfilt.iitb.ac.in/iitb_parallel/iitb_corpus_download/parallel.tgz [following]
--2021-04-20 15:01:39--  https://www.cfilt.iitb.ac.in/iitb_parallel/iitb_corpus_download/parallel.tgz
Connecting to www.cfilt.iitb.ac.in (www.cfilt.iitb.ac.in)|103.21.127.134|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.cfilt.iitb.ac.in/~parallelcorp/iitb_en_hi_parallel/iitb_corpus_download/parallel.tgz [following]
--2021-04-20 15:01:40--  https://www.cfilt.iitb.ac.in/~parallelcorp/iitb_en_hi_parallel/iitb_corpus_download/parallel.tgz
Reusing existing connection to www.cfilt.iitb.ac.in:443.
HTTP request sent, awaiting respo

In [ ]:
import tarfile
my_tar = tarfile.open('parallel.tgz')
my_tar.extractall('.') 
my_tar.close()

In [ ]:
data = []
with open("parallel/IITB.en-hi.en") as f2, open("parallel/IITB.en-hi.hi") as f1:
    for src, tgt in zip(f1, f2):
      data.append(
          {
              "translation": {
                  "hi": src.strip(),
                  "en": tgt.strip()
              }
          }
      )
print(f'total size of data is {len(data)}')

total size of data is 1609682


In [ ]:
data

[{'translation': {'en': 'Give your application an accessibility workout',
   'hi': 'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें'}},
 {'translation': {'en': 'Accerciser Accessibility Explorer',
   'hi': 'एक्सेर्साइसर पहुंचनीयता अन्वेषक'}},
 {'translation': {'en': 'The default plugin layout for the bottom panel',
   'hi': 'निचले पटल के लिए डिफोल्ट प्लग-इन खाका'}},
 {'translation': {'en': 'The default plugin layout for the top panel',
   'hi': 'ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका'}},
 {'translation': {'en': 'A list of plugins that are disabled by default',
   'hi': 'उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है'}},
 {'translation': {'en': 'Highlight duration', 'hi': 'अवधि को हाइलाइट रकें'}},
 {'translation': {'en': 'The duration of the highlight box when selecting accessible nodes',
   'hi': 'पहुंचनीय आसंधि (नोड) को चुनते समय हाइलाइट बक्से की अवधि'}},
 {'translation': {'en': 'Highlight border color',
   'hi': 'सीमांत (बोर्डर) के रंग को हाइलाइट करें'}},
 {'translat

In [ ]:
split = 0.2
train_dataset, eval_dataset = random_split(data, lengths=[int((1-split)*len(data))+1, int(split*len(data))])

In [ ]:
train_dataset.dataset

[{'translation': {'en': 'Give your application an accessibility workout',
   'hi': 'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें'}},
 {'translation': {'en': 'Accerciser Accessibility Explorer',
   'hi': 'एक्सेर्साइसर पहुंचनीयता अन्वेषक'}},
 {'translation': {'en': 'The default plugin layout for the bottom panel',
   'hi': 'निचले पटल के लिए डिफोल्ट प्लग-इन खाका'}},
 {'translation': {'en': 'The default plugin layout for the top panel',
   'hi': 'ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका'}},
 {'translation': {'en': 'A list of plugins that are disabled by default',
   'hi': 'उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है'}},
 {'translation': {'en': 'Highlight duration', 'hi': 'अवधि को हाइलाइट रकें'}},
 {'translation': {'en': 'The duration of the highlight box when selecting accessible nodes',
   'hi': 'पहुंचनीय आसंधि (नोड) को चुनते समय हाइलाइट बक्से की अवधि'}},
 {'translation': {'en': 'Highlight border color',
   'hi': 'सीमांत (बोर्डर) के रंग को हाइलाइट करें'}},
 {'translat

In [ ]:
def data_collator(features:list):

  labels = [f["translation"]["en"] for f in features]
  inputs = [f["translation"]["hi"] for f in features]

  batch = tokenizer.prepare_seq2seq_batch(src_texts=inputs, src_lang="hi_IN", tgt_lang="en_XX", tgt_texts=labels, max_length=32, max_target_length=32)

  for k in batch:
     batch[k] = torch.tensor(batch[k])

  return batch

In [ ]:
model =  XLMProphetNetForConditionalGeneration.from_pretrained('microsoft/xprophetnet-large-wiki100-cased')
tokenizer =XLMProphetNetTokenizer.from_pretrained('microsoft/xprophetnet-large-wiki100-cased')

In [ ]:
args = Seq2SeqTrainingArguments(output_dir="/content/output_dir", do_train=True,do_eval=True,evaluation_strategy="epoch",
per_device_train_batch_size=16,per_device_eval_batch_size=16,
learning_rate=5e-5,num_train_epochs=2)

In [ ]:
trainer = Seq2SeqTrainer(model=model, 
                args=args, 
                data_collator=data_collator, 
                train_dataset=train_dataset, 
                eval_dataset=eval_dataset)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


In [ ]:
trainer.save_model("XLM_LANGUAGE")

In [ ]:
from transformers import pipeline

model_id = "/content/vasudevgupta/XLM_LANGUAGE"
translator = pipeline("translation_hi_to_en", model=model_id, tokenizer=model_id)

In [ ]:
inputs = "अंतिम प्रविष्ट घटना को हाइलाइट करो"

translation = translator(inputs, return_text=True)
translation = [t["translation_text"] for t in translation]
print(translation)